<h2 style="color: orange; text-align: center;">Data Preparation</h2>

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.io import savemat
from datetime import datetime, timedelta

<h2 style="color: orange; text-align: center;">Load Bike Sharing Data</h2>

In [2]:
# Directory of historical bike-sharing files
base_path = r"data_source/bikesharing_monthly"

# List of files with pathlib
bike_files = [f"{base_path}/2023{i:02d}.csv" for i in range(1, 13)]

# Load All Files into a Single DataFrame
bike_data = pd.concat([pd.read_csv(file) for file in bike_files], ignore_index=True)

# Convert Time Columns to Datetime
bike_data['started_at'] = pd.to_datetime(bike_data['started_at'])
bike_data['ended_at'] = pd.to_datetime(bike_data['ended_at'])

# Filter Data for 2023
bike_data = bike_data[bike_data['started_at'].dt.year == 2023]

In [3]:
bike_data.head(5)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,65F0ACD101BF0D49,classic_bike,2023-01-04 19:34:07,2023-01-04 19:39:29,East Falls Church Metro / Sycamore St & 19th St N,31904.0,W Columbia St & N Washington St,32609.0,38.885321,-77.156427,38.885621,-77.166917,member
1,D75158CE73DC43F0,classic_bike,2023-01-27 15:26:38,2023-01-27 19:21:36,Carroll & Westmoreland Ave,32025.0,Fenton St & Ellsworth Dr,32036.0,38.975000,-77.011210,38.997033,-77.025608,member
2,33E85889625FF7CA,classic_bike,2023-01-05 20:44:38,2023-01-05 20:51:18,15th & L St NW,31276.0,Thomas Circle,31241.0,38.903649,-77.034918,38.905900,-77.032500,member
3,E1F055A1651F47A1,classic_bike,2023-01-03 17:45:14,2023-01-03 17:57:23,Hartland Rd & Harte Pl,32255.0,Merrifield Cinema & Merrifield Town Center,32235.0,38.878601,-77.222808,38.870093,-77.229970,member
4,88CC90CEEC298BAF,classic_bike,2023-01-03 05:18:46,2023-01-03 05:25:50,Merrifield Cinema & Merrifield Town Center,32235.0,Hartland Rd & Harte Pl,32255.0,38.870093,-77.229970,38.878601,-77.222808,member


<h2 style="color: orange; text-align: center;">Cleaning</h2>

In [4]:
bike_data = bike_data.dropna(subset=['start_station_id', 'start_station_name', 'end_station_id', 'end_station_name'])

bike_data = bike_data[bike_data['rideable_type'] == 'classic_bike']


<h2 style="color: orange; text-align: center;">Daily Rental Counts</h2>

In [5]:
# Create a Column for the Day
bike_data['day'] = bike_data['started_at'].dt.date

# Daily Rental Counts per Station
daily_pickups = bike_data.groupby(['start_station_id', 'day']).size().unstack(fill_value=0)

<h2 style="color: orange; text-align: center;">Average Daily Trip Duration</h2>

In [ ]:
# Calculate Trip Duration in Seconds
bike_data['duration'] = (bike_data['ended_at'] - bike_data['started_at']).dt.total_seconds()
bike_data = bike_data[bike_data['duration'] >= 120] # The records under two minutes do not make sense
# Average Daily Trip Duration per Station
daily_duration = bike_data.groupby(['start_station_id', 'day'])['duration'].mean().unstack(fill_value=0)

<h2 style="color: orange; text-align: center;">Counts by User Type (Member/Casual)</h2>

In [7]:
# Daily Counts by User Type
daily_member = bike_data[bike_data['member_casual'] == 'member'].groupby(['start_station_id', 'day']).size().unstack(fill_value=0)
daily_casual = bike_data[bike_data['member_casual'] == 'casual'].groupby(['start_station_id', 'day']).size().unstack(fill_value=0)

<h2 style="color: orange; text-align: center;">Load Weather Data</h2>

In [8]:
base_path_weather = Path("data_source/weather_data")

# Load Meteorological Data
file_name = "Washington, DC, United St... 2023-01-01 to 2023-12-31.csv"
weather_data = pd.read_csv(base_path_weather / file_name)

# Convert the Time Column to Datetime
weather_data['datetime'] = pd.to_datetime(weather_data['datetime'])

# Filter Data for 2023
weather_data = weather_data[weather_data['datetime'].dt.year == 2023]

# Select Relevant Columns
weather_data = weather_data[['datetime', 'temp', 'feelslike', 'humidity', 'precip', 'windspeed', 'cloudcover', 'visibility', 'uvindex']]

In [9]:
weather_data.head(5)

,datetime,temp,feelslike,humidity,precip,windspeed,cloudcover,visibility,uvindex
0,2023-01-01,11.0,10.7,82.2,0.101,14.2,47.7,14.6,4
1,2023-01-02,10.5,10.1,79.3,0.000,14.7,87.8,15.9,4
2,2023-01-03,15.2,15.2,78.5,0.000,28.5,90.8,15.6,2
3,2023-01-04,15.3,15.3,85.4,0.000,22.8,85.2,15.9,2
4,2023-01-05,13.5,13.3,70.4,0.000,20.2,75.8,15.9,4


<h2 style="color: orange; text-align: center;">Create the Daily Calendar</h2>

In [10]:
# Create a Daily Calendar for 2023
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 31)
daily_calendar = pd.date_range(start_date, end_date, freq='D')

<h2 style="color: orange; text-align: center;">Create the Binary Array for Weekends</h2>

In [11]:
# Binary Array for Weekends (1 = Weekend, 0 = Weekday)
weekend = (daily_calendar.weekday >= 5).astype(int)

<h2 style="color: orange; text-align: center;">Create the Binary Array for Federal Holidays</h2>

In [12]:
# List of federal holidays in 2023
federal_holidays = [
    datetime(2023, 1, 2),   # New Year's Day
    datetime(2023, 1, 16),  # Dr. Martin Luther King, Jr.'s Birthday
    datetime(2023, 2, 20),  # Washington's Birthday
    datetime(2023, 4, 17),  # D.C. Emancipation Day
    datetime(2023, 5, 29),  # Memorial Day
    datetime(2023, 6, 19),  # Juneteenth National Independence Day
    datetime(2023, 7, 4),   # Independence Day
    datetime(2023, 9, 4),   # Labor Day
    datetime(2023, 10, 9),  # Indigenous Peoples' Day
    datetime(2023, 11, 10), # Veterans Day
    datetime(2023, 11, 23), # Thanksgiving Day
    datetime(2023, 12, 25)  # Christmas Day
]

# Convert federal_holidays into a numpy array of type datetime64[ns]
federal_holidays_np = np.array(federal_holidays, dtype='datetime64[ns]')

# Binary array for holidays (1 = holiday, 0 = non-holiday)
holidays = np.isin(daily_calendar, federal_holidays_np).astype(int)


holidays


array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

<h2 style="color: orange; text-align: center;">Combine Weekends and Holidays</h2>

In [13]:
non_working_days = np.logical_or(weekend, holidays).astype(int)


<h2 style="color: orange; text-align: center;">Daily Data</h2>

In [14]:
# Remove Rows with Problematic Values in Daily_Pickups
daily_pickups_cleaned = daily_pickups.replace([np.inf, -np.inf], np.nan)  
daily_pickups_cleaned = daily_pickups_cleaned.dropna()  
daily_pickups_cleaned = daily_pickups_cleaned.astype(float)
daily_pickups_cleaned[daily_pickups_cleaned == 0] = 1e-6

# Remove Rows with Problematic Values in Daily_Duration
daily_duration_cleaned = daily_duration.replace([np.inf, -np.inf], np.nan)  
daily_duration_cleaned = daily_duration_cleaned.dropna()  
daily_duration_cleaned = daily_duration_cleaned.astype(float)
daily_duration_cleaned[daily_duration_cleaned == 0] = 1e-6

# daily_member
daily_member_cleaned = daily_member.replace([np.inf, -np.inf], np.nan)  
daily_member_cleaned = daily_member_cleaned.dropna()  

# daily_casual
daily_casual_cleaned = daily_casual.replace([np.inf, -np.inf], np.nan)  
daily_casual_cleaned = daily_casual_cleaned.dropna()  

<h2 style="color: orange; text-align: center;">Selecting 100 stations with the most rentals</h2>

In [ ]:
# Copy the original dataframe to avoid modifying it directly
test = daily_member_cleaned.copy()

# Calculate the total rentals for each station
test['total_rentals'] = test.sum(axis=1)

# Sort the stations based on the total rentals in descending order
top_stations_member = test.nlargest(100, 'total_rentals')

# Result: the 100 stations with the most rentals
top_stations_member = top_stations_member.drop(columns='total_rentals')  
top_stations_member


day,2023-01-01,2023-01-02,2023-01-03,2023-01-04,2023-01-05,2023-01-06,2023-01-07,2023-01-08,2023-01-09,2023-01-10,...,2023-12-22,2023-12-23,2023-12-24,2023-12-25,2023-12-26,2023-12-27,2023-12-28,2023-12-29,2023-12-30,2023-12-31
start_station_id,,,,,,,,,,,,,,,,,,,,,
31623.0,19,30,75,107,77,57,37,34,75,89,...,27,11,13,6,21,10,33,21,13,18
31229.0,33,52,47,69,62,60,76,55,56,57,...,32,24,20,9,18,8,36,45,27,35
31201.0,23,38,51,50,56,45,48,56,69,60,...,43,15,17,5,15,16,27,22,22,26
31600.0,23,40,48,53,55,52,38,32,56,60,...,20,16,10,9,9,5,31,23,30,27
31613.0,24,35,37,28,47,33,48,44,54,42,...,25,20,15,7,14,14,32,40,34,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31609.0,15,12,17,13,17,14,16,8,8,7,...,7,6,4,5,0,5,9,12,8,4
31127.0,1,1,3,2,3,3,3,1,3,5,...,3,3,2,2,3,1,2,1,1,2
31239.0,4,10,16,16,15,12,7,3,19,27,...,8,10,7,4,6,9,2,7,6,4


In [26]:
daily_casual_cleaned = daily_casual_cleaned.loc[top_stations_member.index]
daily_member_cleaned = daily_member_cleaned.loc[top_stations_member.index]

In [29]:
daily_pickups_cleaned = daily_pickups_cleaned.loc[top_stations_member.index]
daily_duration_cleaned = daily_duration_cleaned.loc[top_stations_member.index]

In [40]:
# the station IDs of the top 100 stations
top_station_ids = top_stations_member.index

# Group by station ID and take the first occurrence of lat/lon
station_coords = bike_data.groupby('start_station_id')[['start_lat', 'start_lng']].first()

# Filter to keep only the top 100 stations
station_coords_top100 = station_coords.loc[top_station_ids]

# extract lat and lon in the correct order
lat = station_coords_top100['start_lat'].values
lon = station_coords_top100['start_lng'].values

In [41]:
daily_data = {
    'bs_data': {
        'pickups': daily_pickups_cleaned.values,
        'duration': daily_duration_cleaned.values,
        'member': daily_member_cleaned.values,
        'casual': daily_casual_cleaned.values
    },
    'weather_data': {
        'temp': weather_data['temp'].values,
        'feelslike': weather_data['feelslike'].values,
        'humidity': weather_data['humidity'].values,
        'precip': weather_data['precip'].values,
        'windspeed': weather_data['windspeed'].values,
        'cloudcover': weather_data['cloudcover'].values,
        'visibility': weather_data['visibility'].values,
        'uvindex': weather_data['uvindex'].values
    },
    'datetime_calendar': daily_calendar,
    'id_stations': top_station_ids,  
    'lat': lat,  
    'lon': lon,  
    'non_working_days': non_working_days,
    'weekend': weekend,
    'holidays': holidays
}

In [43]:
# Save the Dictionary to a .mat File
savemat('../data_inputs_for_matlab/daily_data.mat', {'daily_data': daily_data})

<h2 style="color: orange; text-align: center;">Checking Results</h2>

In [38]:
print("BS Data Keys:", list(daily_data['bs_data'].keys()))
print("Weather Data Keys:", list(daily_data['weather_data'].keys()))
print("Datetime Calendar Length:", len(daily_data['datetime_calendar']))
print("ID Stations Count:", len(daily_data['id_stations']))


BS Data Keys: ['pickups', 'duration', 'member', 'casual']
Weather Data Keys: ['temp', 'feelslike', 'humidity', 'precip', 'windspeed', 'cloudcover', 'visibility', 'uvindex']
Datetime Calendar Length: 365
ID Stations Count: 100


In [42]:
print("Number of stations in id_stations:", len(daily_data['id_stations']))
print("Shape of lat:", daily_data['lat'].shape)  # Should be (100,)
print("Shape of lon:", daily_data['lon'].shape)  # Should be (100,)
print("Shape of pickups:", daily_data['bs_data']['pickups'].shape)  # Should be (100, 365)

Number of stations in id_stations: 100
Shape of lat: (100,)
Shape of lon: (100,)
Shape of pickups: (100, 365)
